In [1]:
import pandas as pd
import re
import string
import matplotlib.pyplot as plt
import numpy as np
import spacy
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer


In /Users/amybeisel/opt/anaconda3/envs/U4-S1-NLP/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The savefig.frameon rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.
In /Users/amybeisel/opt/anaconda3/envs/U4-S1-NLP/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The verbose.level rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.
In /Users/amybeisel/opt/anaconda3/envs/U4-S1-NLP/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The verbose.fileo rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.


In [2]:
df = pd.read_csv('data/cannabis.csv')

In [3]:
print(df.shape)
df


(2351, 6)


Strain    Type  Rating  \
0             100-Og  hybrid     4.0   
1     98-White-Widow  hybrid     4.7   
2               1024  sativa     4.4   
3           13-Dawgs  hybrid     4.2   
4           24K-Gold  hybrid     4.6   
...              ...     ...     ...   
2346         Zeus-Og  hybrid     4.7   
2347        Zkittlez  indica     4.6   
2348     Zombie-Kush  indica     5.0   
2349       Zombie-Og  indica     4.4   
2350        Zoom-Pie  indica     4.6   

                                         Effects                     Flavor  \
0     Creative,Energetic,Tingly,Euphoric,Relaxed        Earthy,Sweet,Citrus   
1       Relaxed,Aroused,Creative,Happy,Energetic      Flowery,Violet,Diesel   
2      Uplifted,Happy,Relaxed,Energetic,Creative    Spicy/Herbal,Sage,Woody   
3        Tingly,Creative,Hungry,Relaxed,Uplifted  Apricot,Citrus,Grapefruit   
4      Happy,Relaxed,Euphoric,Uplifted,Talkative       Citrus,Earthy,Orange   
...                                          ...                        ...   
2346   Happy,Uplifted,Relaxed,Euphoric,Energetic          Earthy,Woody,Pine   
2347      Relaxed,Happy,Euphoric,Uplifted,Sleepy          Sweet,Berry,Grape   
2348     Relaxed,Sleepy,Talkative,Euphoric,Happy  Earthy,Sweet,Spicy/Herbal   
2349        Relaxed,Sleepy,Euphoric,Happy,Hungry       Sweet,Earthy,Pungent   
2350        Hungry,Relaxed,Uplifted,Happy,Sleepy       Berry,Earthy,Pungent   

                                            Description  
0     $100 OG is a 50/50 hybrid strain that packs a ...  
1     The ‘98 Aloha White Widow is an especially pot...  
2     1024 is a sativa-dominant hybrid bred in Spain...  
3     13 Dawgs is a hybrid of G13 and Chemdawg genet...  
4     Also known as Kosher Tangie, 24k Gold is a 60%...  
...                                                 ...  
2346  Zeus OG is a hybrid cross between Pineapple OG...  
2347  Zkittlez is an indica-dominant mix of Grape Ap...  
2348  Zombie Kush by Ripper Seeds comes from two dif...  
2349  If you’re looking to transform into a flesh-ea...  
2350  Zoom Pie (also known as Zombie Pie) is a heavy...  

[2351 rows x 6 columns]

In [4]:
df['Effects'].isnull().sum()

0

In [5]:
df['Description'].isnull().value_counts()

False    2318
True       33
Name: Description, dtype: int64

In [6]:
df[df['Description'].isnull()]

Strain    Type  Rating  \
163              Banana-Candy  indica     4.2   
368               Boysenberry  hybrid     3.6   
594               Crater-Lake  hybrid     4.6   
679                    Domino  indica     4.0   
736                Earthquake  hybrid     4.5   
817                    Frosty  indica     4.4   
856               Ghost-Bubba  hybrid     4.6   
899                  Goldwing  sativa     4.7   
949              Green-Hornet  hybrid     4.2   
955             Green-Monster  hybrid     3.7   
970                Guido-Kush  indica     4.1   
1012    Hawaiian-Thunder-Fuck  hybrid     4.0   
1101                Jack-Kush  hybrid     4.1   
1199                 Krakatoa  hybrid     4.0   
1289            Key-Lime-Haze  sativa     4.2   
1365                    Matsu  indica     5.0   
1368               Maui-Berry  hybrid     4.0   
1476           Northern-Skunk  hybrid     4.5   
1511  Old-Lyme-Connecticut-Og  hybrid     4.5   
1513            Old-School-Og  hybrid     3.8   
1542         Oregon-Pineapple  hybrid     4.2   
1544           Organic-Diesel  hybrid     4.5   
1633           Platinum-Wreck  hybrid     4.1   
1636              Poison-Haze  sativa     4.1   
1660                  Pure-Og  indica     4.1   
1686          Purple-Elephant  indica     4.2   
1726             Purple-Tonic  indica     3.2   
1801           Root-Beer-Kush  sativa     3.8   
1860              Sierra-Mist  hybrid     4.7   
1993             Starry-Night  hybrid     4.6   
2101           Taliban-Poison  indica     3.7   
2195            Tropical-Tang  hybrid     4.5   
2321           Wonderwoman-Og  indica     4.0   

                                          Effects                      Flavor  \
163      Relaxed,Euphoric,Uplifted,Creative,Happy     Tree,Fruit,Earthy,Sweet   
368   Giggly,Relaxed,Talkative,Energetic,Euphoric          Berry,Sweet,Earthy   
594     Happy,Uplifted,Talkative,Relaxed,Creative         Citrus,Pine,Pungent   
679         Sleepy,Happy,Tingly,Uplifted,Euphoric                         NaN   
736    Creative,Euphoric,Uplifted,Happy,Energetic           Woody,Pine,Earthy   
817          Sleepy,Relaxed,Creative,Hungry,Happy          Earthy,Sweet,Skunk   
856       Happy,Energetic,Uplifted,Relaxed,Giggly         Sweet,Pungent,Grape   
899    Uplifted,Happy,Euphoric,Creative,Energetic                         NaN   
949        Euphoric,Uplifted,Relaxed,Giggly,Happy            Sweet,Woody,Pine   
955          Happy,Relaxed,Uplifted,Hungry,Giggly         Earthy,Chemical,Tar   
970     Euphoric,Uplifted,Creative,Relaxed,Sleepy                         NaN   
1012  Uplifted,Creative,Euphoric,Energetic,Hungry         Earthy,Sweet,Citrus   
1101      Happy,Energetic,Giggly,Euphoric,Relaxed        Sweet,Citrus,Pungent   
1199     Energetic,Euphoric,Sleepy,Uplifted,Happy                         NaN   
1289    Happy,Uplifted,Euphoric,Energetic,Relaxed           Lime,Citrus,Sweet   
1365          Euphoric,Tingly,Happy,Sleepy,Hungry                         NaN   
1368     Happy,Creative,Uplifted,Hungry,Energetic        Berry,Citrus,Pungent   
1476       Happy,Relaxed,Euphoric,Uplifted,Hungry        Skunk,Earthy,Pungent   
1511  Energetic,Happy,Talkative,Uplifted,Euphoric     Tree,Fruit,Pungent,Pine   
1513         Sleepy,Relaxed,Tingly,Happy,Euphoric        Pungent,Skunk,Earthy   
1542       Happy,Giggly,Uplifted,Relaxed,Creative      Pineapple,Sweet,Cheese   
1544   Uplifted,Happy,Talkative,Energetic,Focused      Diesel,Pungent,Tobacco   
1633     Happy,Uplifted,Euphoric,Relaxed,Creative           Earthy,Pine,Sweet   
1636      Energetic,Happy,Relaxed,Uplifted,Giggly  Citrus,Orange,Spicy/Herbal   
1660       Happy,Tingly,Uplifted,Creative,Relaxed         Earthy,Pine,Flowery   
1686         Relaxed,Sleepy,Giggly,Happy,Euphoric          Grape,Earthy,Woody   
1726     Happy,Uplifted,Energetic,Euphoric,Sleepy  Blueberry,Nutty,Tree,Fruit   
1801   Relaxed,Uplifted,Energetic,Giggly,Creative       Sweet,Vanilla,Pungent   
1860   

In [7]:
df = df.replace('None', np.nan)

In [8]:
df = df.dropna().reset_index()

In [9]:
len(df['Description'])

2163

In [10]:
df.shape

(2163, 7)

In [11]:
df["Description"].dtype

dtype('O')

In [12]:
#df['Description'] = df['Description'].apply(str).str.replace('.', ',')

In [13]:
df["Effects"][0]

'Creative,Energetic,Tingly,Euphoric,Relaxed'

In [14]:
import spacy
from spacy.tokenizer import Tokenizer
from sklearn.neighbors import NearestNeighbors
from sklearn.feature_extraction.text import TfidfVectorizer

nlp = spacy.load("en_core_web_lg")
df.head()

# The Tokenizer
tokenizer = Tokenizer(nlp.vocab)

# Make the tokens for description
description_tokens = []
for txt in tokenizer.pipe(df['Effects'], batch_size=500):
    txt_tokens = [token.text for token in txt]
    description_tokens.append(txt_tokens)
df['description_tokens'] = description_tokens
print(df['description_tokens'].head())



0    [Creative,Energetic,Tingly,Euphoric,Relaxed]
1      [Relaxed,Aroused,Creative,Happy,Energetic]
2     [Uplifted,Happy,Relaxed,Energetic,Creative]
3       [Tingly,Creative,Hungry,Relaxed,Uplifted]
4     [Happy,Relaxed,Euphoric,Uplifted,Talkative]
Name: description_tokens, dtype: object


In [15]:
df.head()

index          Strain    Type  Rating  \
0      0          100-Og  hybrid     4.0   
1      1  98-White-Widow  hybrid     4.7   
2      2            1024  sativa     4.4   
3      3        13-Dawgs  hybrid     4.2   
4      4        24K-Gold  hybrid     4.6   

                                      Effects                     Flavor  \
0  Creative,Energetic,Tingly,Euphoric,Relaxed        Earthy,Sweet,Citrus   
1    Relaxed,Aroused,Creative,Happy,Energetic      Flowery,Violet,Diesel   
2   Uplifted,Happy,Relaxed,Energetic,Creative    Spicy/Herbal,Sage,Woody   
3     Tingly,Creative,Hungry,Relaxed,Uplifted  Apricot,Citrus,Grapefruit   
4   Happy,Relaxed,Euphoric,Uplifted,Talkative       Citrus,Earthy,Orange   

                                         Description  \
0  $100 OG is a 50/50 hybrid strain that packs a ...   
1  The ‘98 Aloha White Widow is an especially pot...   
2  1024 is a sativa-dominant hybrid bred in Spain...   
3  13 Dawgs is a hybrid of G13 and Chemdawg genet...   
4  Also known as Kosher Tangie, 24k Gold is a 60%...   

                             description_tokens  
0  [Creative,Energetic,Tingly,Euphoric,Relaxed]  
1    [Relaxed,Aroused,Creative,Happy,Energetic]  
2   [Uplifted,Happy,Relaxed,Energetic,Creative]  
3     [Tingly,Creative,Hungry,Relaxed,Uplifted]  
4   [Happy,Relaxed,Euphoric,Uplifted,Talkative]

In [16]:
def tokenize(document):
    
    doc = nlp(document)
    
    return [token.lemma_.strip() for token in doc if (token.is_stop != True) and (token.is_punct != True)]

In [17]:

# Instantiate vectorizer object
tfidf = TfidfVectorizer(stop_words = 'english',
                       #ngram_range = (1,2),
                       max_features = 5000)

# Create a vocabulary and tf-idf score per document
dtm = tfidf.fit_transform(df['Effects'])
                         

# Get feature names to use as dataframe column headers
dtm = pd.DataFrame(dtm.todense(), columns=tfidf.get_feature_names())
nn = NearestNeighbors(n_neighbors=4, algorithm='kd_tree')
nn.fit(dtm)


# View Feature Matrix as DataFrame
print(dtm.shape)
dtm.head()

(2163, 15)


aroused  creative  dry  energetic  euphoric  focused  giggly     happy  \
0  0.000000  0.458104  0.0   0.492003  0.285453      0.0     0.0  0.000000   
1  0.694405  0.428473  0.0   0.460180  0.000000      0.0     0.0  0.239493   
2  0.000000  0.553938  0.0   0.594929  0.000000      0.0     0.0  0.309622   
3  0.000000  0.441144  0.0   0.000000  0.000000      0.0     0.0  0.000000   
4  0.000000  0.000000  0.0   0.000000  0.343135      0.0     0.0  0.307797   

     hungry  mouth   relaxed  sleepy  talkative    tingly  uplifted  
0  0.000000    0.0  0.272771     0.0   0.000000  0.626248  0.000000  
1  0.000000    0.0  0.255128     0.0   0.000000  0.000000  0.000000  
2  0.000000    0.0  0.329834     0.0   0.000000  0.000000  0.366819  
3  0.536072    0.0  0.262672     0.0   0.000000  0.603063  0.292126  
4  0.000000    0.0  0.327890     0.0   0.739617  0.000000  0.364657

In [18]:
dtm.columns

Index(['aroused', 'creative', 'dry', 'energetic', 'euphoric', 'focused',
       'giggly', 'happy', 'hungry', 'mouth', 'relaxed', 'sleepy', 'talkative',
       'tingly', 'uplifted'],
      dtype='object')

In [19]:
ideal = ["""
Creative,Uplifted,Tingly,Euphoric,Relaxed, Giggly
"""]

In [20]:
# Query the ideal descprition
new = tfidf.transform(ideal)
new

<1x15 sparse matrix of type '<class 'numpy.float64'>'
	with 6 stored elements in Compressed Sparse Row format>

In [21]:
pred = nn.kneighbors(new.todense())
pred

(array([[0.43079886, 0.43079886, 0.53743649, 0.54335812]]),
 array([[1058,  853,   84,  129]]))

In [22]:
recommended = pred[1]
recommended

array([[1058,  853,   84,  129]])

In [23]:
# most ideal description,strain and flavor
df['Description'][1058]

'Not much is known about the California-bred hybrid Jolly Rancher, but this hybrid strain has nevertheless carved a name for itself among West Coast genetics. Its name and initial appeal lies primarily in Jolly Rancher’s sour candied aroma, although there’s also something to be said about its high-flying euphoria and colorful buds. Coated with a sticky blanket of sugar-like resin, Jolly Rancher is a treat for all the senses.'

In [24]:
print(df['Strain'][1058])
print(df['Effects'][1058])
print(df['Flavor'][1058])
print(df['Rating'][1058])

Jolly-Rancher
Euphoric,Happy,Giggly,Tingly,Creative
Sweet,Pungent,Berry
4.3


In [25]:
#to be pickled....
#nn, dtm, tfidf

In [26]:
import pickle
# Dump the trained classifier (nn)  with Pickle
pickle_filename = 'model.pkl'
pickled_model = open(pickle_filename, 'wb')  # Open the file to save as pkl file
pickle.dump(nn, pickled_model)
pickled_model.close() # Close the pickle instances


In [27]:
# Loading the saved model
model_pkl = open(pickle_filename, 'rb')
NN_model = pickle.load(model_pkl)
print ("Loaded model :: ", NN_model)  # print to verify

Loaded model ::  NearestNeighbors(algorithm='kd_tree', leaf_size=30, metric='minkowski',
                 metric_params=None, n_jobs=None, n_neighbors=4, p=2,
                 radius=1.0)


In [28]:
# Dump the trained classifier (tfidf)  with Pickle
pickle_filename_1 = 'tfidf.pkl'
pickled_model_1 = open(pickle_filename_1, 'wb')  # Open the file to save as pkl file
pickle.dump(tfidf, pickled_model_1)
pickled_model_1.close() # Close the pickle instances

In [29]:
# Loading the saved model
model_pkl_1 = open(pickle_filename_1, 'rb')
tfidf_model = pickle.load(model_pkl_1)
print ("Loaded model :: ", tfidf_model)  # print to verify

Loaded model ::  TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=5000,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words='english', strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)


In [30]:
dir(NN_model)

['__abstractmethods__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_check_algorithm_metric',
 '_fit',
 '_fit_X',
 '_fit_method',
 '_get_param_names',
 '_get_tags',
 '_kneighbors_reduce_func',
 '_more_tags',
 '_pairwise',
 '_radius_neighbors_reduce_func',
 '_tree',
 'algorithm',
 'effective_metric_',
 'effective_metric_params_',
 'fit',
 'get_params',
 'kneighbors',
 'kneighbors_graph',
 'leaf_size',
 'metric',
 'metric_params',
 'n_jobs',
 'n_neighbors',
 'p',
 'radius',
 'radius_neighbors',
 'radius_neighbors_graph',
 'set_params']

In [31]:
ideal2 = ["happy,Sleepy,Apricot,Citrus,Grapefruit"]

In [32]:
new2 = tfidf_model.transform(ideal2)

In [33]:
pred1 = NN_model.kneighbors(new2.todense())
pred1

(array([[0.47515218, 0.63432626, 0.63432626, 0.63432626]]),
 array([[ 684, 1831, 1118,  894]]))

In [34]:
print(df['Strain'][684])
print(df['Flavor'][684])
print(df['Effects'][684])
print(df['Rating'][684])

Eastern-European
Earthy,Woody,Pine
Sleepy,Happy,Relaxed
5.0


In [35]:
recommended1 = pred1[1]
recommended1

array([[ 684, 1831, 1118,  894]])

In [36]:
import json
def recommend(user_input):
    temp_df = NN_model.kneighbors(tfidf_model.transform([user_input]).todense())[1]
    

    #print(temp_df)
    
    for i in range(4):
        info = df.iloc[temp_df[0][i]]['Strain']
        info_effects = df.iloc[temp_df[0][i]]['Effects']
        info_flavor = df.iloc[temp_df[0][i]]['Flavor']
        info_description = df.iloc[temp_df[0][i]]['Description']
        info_rating = df.iloc[temp_df[0][i]]['Rating']
        
        print(json.dumps(info))
        print(json.dumps(info_effects))
        print(json.dumps(info_flavor))
        print(json.dumps(info_description))
        print(json.dumps(info_rating))
        
        #return json.dumps(info) #for engineeers, the return does not work in jupyter lab.  Should work in vsCode. 
        #return json.dumps(info_effects)
        #return json.dumps(info_flavor)
        #return json.dumps(info_description)
        #return json.dumps(info_rateing)

In [41]:
df.iloc[668]

index                                                               723
Strain                                                       Dutch-Haze
Type                                                             sativa
Rating                                                                4
Effects                        Relaxed,Energetic,Creative,Happy,Aroused
Flavor                                                    Citrus,Earthy
Description           Dutch Haze by Dutch Passion is a 90% sativa-do...
description_tokens           [Relaxed,Energetic,Creative,Happy,Aroused]
Name: 668, dtype: object

In [38]:
dtm.columns

Index(['aroused', 'creative', 'dry', 'energetic', 'euphoric', 'focused',
       'giggly', 'happy', 'hungry', 'mouth', 'relaxed', 'sleepy', 'talkative',
       'tingly', 'uplifted'],
      dtype='object')

In [39]:
#Uplifted,Happy,Relaxed,Energetic,Creative = Keyerror: 1633
#Relaxed,Aroused,Creative,Happy,Energeti = Keyerror: 668
#Relaxed,Tingly,Euphoric,Aroused,Happy = Keyerror: 695

recommend('hybrid-like in its balanced calm and moderate cerebral effects')

"Club-69"
"Sleepy,Relaxed"
"Diesel,Flowery,Earthy"
"Club 69 is a special strain crafted by Josh D of Karma Genetics for Arizona\u2019s own, Sunday Goods. This Kushy cross of Karmarado OG and Biker Kush is a knockout for consumers looking for a heavy, restful indica-dominant strain. The terpene profile is a mixture of gas and pine, leaving the consumer with clean floral notes on the exhale. Enjoy this strain with care as the effects have been known to be powerfully sedative with continued consumption."
5.0
"Dawgfather-Og"
"Sleepy,Relaxed"
"Spicy/Herbal,Pepper,Pungent"
"Dawgfather OG, or simply \u201cThe Dawgfather,\u201d is an indica-dominant hybrid created by AlphaKronik Genes capable of massive plant production and a healthier-than-average yield. With potency floating above 20%, this strain contends well with chronic pain, insomnia, Crohn's, and anxiety. Expect traditional notes of gas/chemicals, Kush, and some funkiness. Dawgfather OG offers a strong buzz with nuance worth exploring.